# YOLOv10 + Weights & Biases (W&B) - Custom Dataset Training

This notebook demonstrates how to train a YOLOv10 model on a custom dataset using Weights & Biases (W&B) for experiment tracking. The dataset is imported from Roboflow and used in combination with YOLOv10 to detect objects relevant to your project.

We also perform a grid search over multiple hyperparameters (model version, batch size, epochs, optimizer) to find the best configuration for training. After training, we can visualize the results (e.g., loss, mAP, precision, recall) directly in Weights & Biases for analysis and comparison.

---

**Author**: Agus Darmawan  
**GitHub**: [github.com/agus-darmawan](https://github.com/agus-darmawan)


### Check GPU Availability

Before starting, we check if a GPU is available for training. This is crucial for optimizing training speed and performance.

In [ ]:
!nvidia-smi

### Initialize Project Directory

We set up the main working directory for our YOLOv10 project. This includes creating a directory for datasets if it doesn't already exist.

In [ ]:
from pathlib import Path
HOME = Path.cwd() / "yolov10"
DATASETS_PATH = HOME / "datasets"
DATASETS_PATH.mkdir(parents=True, exist_ok=True)
print(f"📁 Project directory: {HOME}")

### Utility: Install Required Dependencies

This utility function checks if a package is installed, and if not, installs it using pip. This ensures that all necessary packages are available for the notebook to run smoothly.

In [ ]:
import sys, subprocess, importlib
def ensure_package_installed(package_name, import_name=None, version_spec=""):
    import_name = import_name or package_name
    try:
        importlib.import_module(import_name)
        print(f"✅ {import_name} is already installed")
    except ImportError:
        print(f"❌ {import_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package_name}{version_spec}"])
        print(f"✅ {package_name} installed")

### Install and Verify YOLOv10

We install the YOLOv10 package and verify the installation by checking the version. This ensures that we are using the correct version of YOLOv10 for our training.

In [ ]:
ensure_package_installed("ultralytics", version_spec=">=10,<11")
from ultralytics import YOLO
import ultralytics
ultralytics.checks()

### Optional: Test Initial Inference

We can run a quick inference test using a sample image to ensure that YOLOv10 is working correctly. This step is optional but recommended to confirm the installation.

In [ ]:
from IPython.display import Image, display
%cd {HOME}
!yolo task=detect mode=predict model=yolov10n.pt conf=0.25 source='/img/tes.jpg' save=True
display(Image(filename='runs/detect/predict/tes.jpg', height=600))

### Setup Roboflow API & Environment Variables

We set up the Roboflow API key and load it from an environment variable. This key is essential for accessing datasets hosted on Roboflow.

In [ ]:
ensure_package_installed("python-dotenv", "dotenv")
ensure_package_installed("roboflow")

import os
from dotenv import load_dotenv
from roboflow import Roboflow

load_dotenv()

api_key = os.getenv("ROBOFLOW_API_KEY")
if not api_key:
    raise ValueError("❌ API key not found. Ensure .env file contains ROBOFLOW_API_KEY.")
print("✅ API key loaded successfully")

### Download Dataset from Roboflow

We download the dataset from Roboflow using the API key. This step fetches the dataset and prepares it for training.

In [ ]:
os.chdir(DATASETS_PATH)

rf = Roboflow(api_key=api_key)
project = rf.workspace("agus-darmawan").project("subtantion-monitoring")
version = project.version(1)
dataset = version.download("yolov10")

## Setup Weights & Biases (W&B)

Weights & Biases (W&B) is a powerful tool for experiment tracking and visualization. We will set up W&B to log our training metrics and hyperparameters.

In [ ]:
ensure_package_installed("wandb")
ensure_package_installed("torch")

import wandb
import torch

### Train Function with W&B Logging

This function encapsulates the training process with YOLOv10 and logs all relevant metrics and hyperparameters to Weights & Biases.

In [ ]:
def train_yolo_with_wandb(version, epoch, batch_size, optimizer, dataset_location):
    base_dir = HOME / f"experiments/yolov10{version}"
    experiment_name = f"epoch_{epoch}_batch_{batch_size}_optimizer_{optimizer}"
    experiment_path = base_dir / experiment_name
    experiment_path.mkdir(parents=True, exist_ok=True)

    wandb.init(
        project="electrical-substation-detection",
        entity="dar-ta",
        name=f"yolov10{version}-{experiment_name}",
        config={
            "epochs": epoch,
            "batch_size": batch_size,
            "optimizer": optimizer,
            "version": version
        },
        settings=wandb.Settings(init_timeout=300),
        dir=str(experiment_path)
    )

    model = YOLO(f"yolov10{version}.pt")
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model.train(
        data=f"{dataset_location}/data.yaml",
        epochs=epoch,
        batch=batch_size,
        optimizer=optimizer,
        weight_decay=0.0005,
        warmup_epochs=3,
        save_dir=str(experiment_path),
        device=device
    )

    wandb.finish()

### Run Grid Search for Multiple Experiments

We perform a grid search over various hyperparameters to find the best configuration for our YOLOv10 model. This includes different versions, epochs, batch sizes, and optimizers.

In [ ]:
dataset_location = str(DATASETS_PATH / "subtantion-monitoring-1")
versions = ["n", "s", "m", "l", "x"]
epochs = [50, 100,150, 200]
batch_sizes = [2, 4, 8, 16]
optimizers = ["Adam", "SGD"]

for version in versions:
    for epoch in epochs:
        for batch_size in batch_sizes:
            for optimizer in optimizers:
                train_yolo_with_wandb(version, epoch, batch_size, optimizer, dataset_location)